In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import cv2
import glob
from time import time
import pandas as pd

import numpy as np
import keras
from keras.utils import Sequence
from keras.models import Sequential, Model 
from keras import layers
from keras.callbacks import TensorBoard
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import os
import tensorflow as tf

from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D

In [0]:
path = "/content/drive/My Drive/Affective Computing/Assn_3/"

In [0]:
# Create the model
def emotion_recognition_model():
  model = Sequential()
  model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(120,224,224,3)))
  model.add(MaxPooling3D(pool_size=(2, 2, 2)))
  model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
  model.add(MaxPooling3D(pool_size=(2, 2, 2)))
  model.add(Conv3D(96, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
  model.add(MaxPooling3D(pool_size=(2, 2, 2)))
  model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
  model.add(MaxPooling3D(pool_size=(2, 2, 2)))
  model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
  model.add(MaxPooling3D(pool_size=(2, 2, 2)))
  model.add(Flatten())
  model.add(Dense(2048, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(7, activation='softmax'))
  return model

In [13]:
model = emotion_recognition_model()
model.summary() 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 118, 222, 222, 32) 2624      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 59, 111, 111, 32)  0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 57, 109, 109, 64)  55360     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 28, 54, 54, 64)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 26, 52, 52, 96)    165984    
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 13, 26, 26, 96)    0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 11, 24, 24, 128)  

In [0]:
# class for inputing data batch by batch
#Not generalized enough works only for batch size 1 as of now i.e. (1,120,224,224,3) which is required for doing 3D convolution and detect optical flow to determine a given action
class MY_Generator(Sequence):
    def __init__(self, video_filenames, video_gt, batch_size):
        self.video_filenames, self.video_gt = video_filenames, video_gt
        self.batch_size = batch_size

    def __len__(self):
        return len(self.video_filenames) 

    def __getitem__(self, idx):
        batch_x = self.video_filenames[idx:idx+self.batch_size]
        batch_y = self.video_gt[idx:idx+self.batch_size]
        x = load_img(batch_x[0])
        x = img_to_array(x)
        video = np.zeros((self.batch_size,120,224,224,3))
        # y = cv2.imread(batch_y[0])
        for i in range(self.batch_size):
          for j in range(120):
            video[i,j,:,:,:] = x[224*j:224*(j+1),:,:]
        video /= 255 
        label = batch_y[0]
        gt = np.zeros((self.batch_size,7))
        for i in range(self.batch_size):
          if label == 'happy':
            gt[i,:] = [0,0,0,0,0,0,0]
          elif label == 'sad':
            gt[i,:] = [1,0,0,0,0,0,0]
          elif label == 'disgust':
            gt[i,:] = [0,1,0,0,0,0,0]
          elif label == 'angry':
            gt[i,:] = [0,0,1,0,0,0,0]
          elif label == 'surprised':
            gt[i,: ] = [0,0,0,1,0,0,0]
          elif label == 'calm':
            gt[i,:] = [0,0,0,0,1,0,0]
          elif  label == 'neutral':
            gt[i,:] = [0,0,0,0,0,1,0]
          else:
            gt[i,:] = [0,0,0,0,0,0,1]
        # print(image,gt)
        return video,gt

In [17]:
def main():
  
  model = emotion_recognition_model()
  adam = keras.optimizers.Adam(lr=0.001)
  model.compile(loss=keras.losses.categorical_crossentropy, optimizer=adam, metrics=['accuracy'])
  csv_file = pd.read_csv(path+'train_label.csv')
  z = csv_file.iloc[:,0].values
  gt = csv_file.iloc[:,1].values
  #print(z.head())
  print(z.shape)
  final_index = len(glob.glob(path+'data_preprocessing/*'))
  print(final_index)

  csv_test_file = pd.read_csv(path+'test_label.csv')
  test_z = csv_test_file.iloc[:,0].values
  gt_test = csv_test_file.iloc[:,1].values
  #print(z.head())
 # print(z.shape)
  test_final_index = len(glob.glob(path+'test_data_proc/*'))


  # print(final_index)
  initial_index = 1
  # final_index = 10
  #print(final_index)
  train_video_filenames = []
  train_video_filenames[:] = z[:]
  # print(train_video_filenames)
  train_video_ground_truth = []
  train_video_ground_truth = gt[:]
  for i in range(len(train_video_filenames)):
    train_video_filenames[i] = path + train_video_filenames[i][2:]
  

  validate_video_filenames = []
  validate_video_filenames[:] = test_z[:]
  for i in range(len(validate_video_filenames)):
    validate_video_filenames[i] = path + validate_video_filenames[i][2:]

  validate_video_ground_truth = []
  validate_video_ground_truth = gt_test[:]

  #creating my_generator objects
  batch_size = 1
  my_training_batch_generator = MY_Generator(train_video_filenames,train_video_ground_truth, batch_size)
  my_validation_batch_generator = MY_Generator(validate_video_filenames, validate_video_ground_truth, batch_size)


  # os.mkdir(path+'checkpoints')
  # os.mkdir(path+'logs')
  checkpoint_path = path+"checkpoints/cp-1.ckpt"
  cp_callback = keras.callbacks.ModelCheckpoint(checkpoint_path,monitor='val_loss',
     verbose=1, save_best_only=True,
     mode = 'min')
  tensorboard = TensorBoard(log_dir=path+"logs/{}".format(time()))  
  #training started
  model.fit_generator(generator=my_training_batch_generator,
                                         steps_per_epoch=(len(train_video_filenames)),
                                         epochs=6,
                                         verbose=1,
                                         validation_data=my_validation_batch_generator,
                                         validation_steps=(len(validate_video_filenames)),
                                         callbacks=[tensorboard,cp_callback])
  print("successfully finished training!!!")

main()


(2400,)
2400
Epoch 1/6
2400/2400 [==============================] - 3688s 2s/step - loss: 1.7808 - accuracy: 0.1704 - val_loss: 2.2420 - val_accuracy: 0.1333

Epoch 00001: val_loss improved from inf to 2.24201, saving model to /content/drive/My Drive/Affective Computing/Assn_3/checkpoints/cp-1.ckpt
Epoch 2/6
2400/2400 [==============================] - 1452s 605ms/step - loss: 1.9328 - accuracy: 0.1492 - val_loss: 1.4523 - val_accuracy: 0.1333

Epoch 00002: val_loss improved from 2.24201 to 1.45229, saving model to /content/drive/My Drive/Affective Computing/Assn_3/checkpoints/cp-1.ckpt
Epoch 3/6
2400/2400 [==============================] - 1448s 603ms/step - loss: 2.1151 - accuracy: 0.1663 - val_loss: 3.5250 - val_accuracy: 0.2667

Epoch 00003: val_loss did not improve from 1.45229
Epoch 4/6
2400/2400 [==============================] - 1443s 601ms/step - loss: 2.3513 - accuracy: 0.1513 - val_loss: 2.8661 - val_accuracy: 0.1333

Epoch 00004: val_loss did not improve from 1.45229
Epoch 

2400
2400
480
480
